## 4-6. TF-IDF로 성능을 높여보자
카운트 벡터에서 빈도는 일종의 가중치로 작용한다. 즉, 카운트 벡터에서는 빈도가 높을수록 중요한 단어로 취급되는 경향이 있다. 그러나 여기에는 한 가지 문제가 있는데, 만일 어떤 단어가 모든 문서에 나타난다면 그 단어는 과연 중요한 단어일까? 예를 들어 가상의 축구팀들을 만들어 가장 강한 팀을 가린다고 할 때, 모든 팀에 3명의 같은 선수가 똑같이 들어있다면 이 3명의 선수가 팀 간의 차이를 만들 수 있을까? 당연히 아니라고 생각할 것이며, 모든 문서에 다 들어있는 단어 또한 별로 중요하지 않다. 이것을 정도로 바꿔서 이야기한다면, 단어가 더 많은 문서에서 나타날수록 그 단어는 별로 중요하지 않게 된다. 이러한 의미를 카운트 벡터에 반영한 것이 바로 'TF-IDF(Term Frequency - Inverse Document Frequency)'다. 이를 용어 그대로 햇헉하면 '단어빈도-역문서빈도'인데, 이것이 의미하는 것은 카운트 대신 단어의 빈도에 그 단어가 출현한 문서 수의 역수를 곱했다는 뜻이다. 다시 말해서 단어의 빈도를 그 단어가 나타난 문서의 수로 나눠준 것이다. 이렇게 하면 단어가 나타난 문서의 수가 클수록 이 값은 작아져서 우리 목적을 달성하게 된다.

TF-IDF에는 다양한 변형식이 있으므로, 의미만 잘 이해하고 라이브러리를 이용하면 편하다. 사이킷런은 텍스트로부터 바로 TF-IDF 행렬을 생성하는 [TfidfVectorizer 클래스](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)와 카운트 벡터로부터 변환하는 [TfidfTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html)를 함께 제공한다.

다음 예시에서는 앞서 만든 카운트 행렬을 재사용하기 위해 TfidfTransformer를 사용한다. 이렇게 하면 시간을 절약할 수 있는데, 처음부터 TF-IDF 행렬을 만들고 싶다면 TfidfVectorizer를 쓰면 된다.

In [1]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
transformer

TfidfTransformer()

In [2]:
from nltk.corpus import movie_reviews
reviews = [movie_reviews.raw(fileid) for fileid in movie_reviews.fileids()]


from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
tokenizer = RegexpTokenizer("[\w']{3,}")
english_stops = set(stopwords.words('english'))

documents = [movie_reviews.raw(fileid) for fileid in movie_reviews.fileids()]

tokens = [[token for token in tokenizer.tokenize(doc) if token not in english_stops] for doc in documents]

word_count = {}
for text in tokens:
    for word in text:
        word_count[word] = word_count.get(word, 0) + 1

sorted_features = sorted(word_count, key=word_count.get, reverse=True)

word_features = sorted_features[:1000]


from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(vocabulary=word_features)
reviews_cv = cv.fit_transform(reviews)

In [6]:
reviews_tfidf = transformer.fit_transform(reviews_cv)

# TF-IDF 행렬의 모양과 카운트 행렬의 모양이 일치하는 것을 확인
print("# shape of tfidf matrix:", reviews_tfidf.shape)
print()

# 첫 리뷰의 카운트 벡터 중 앞 20개 값 출력
print('# 20 count score of the first review:', reviews_cv[0].toarray()[0][:20]) 
print()

# 첫 리뷰의 TF-IDF 벡터 중 앞 20개 값 출력
print('# 20 tfidf score of the first review:', reviews_tfidf[0].toarray()[0][:20])

# shape of tfidf matrix: (2000, 1000)

# 20 count score of the first review: [6 3 6 3 3 0 2 0 1 0 1 3 2 2 1 0 1 2 3 5]

# 20 tfidf score of the first review: [0.13556199 0.06700076 0.14998642 0.0772298  0.08608998 0.
 0.0609124  0.         0.03126552 0.         0.03242315 0.09567082
 0.06575035 0.06518293 0.03225625 0.         0.0345017  0.06863314
 0.10042383 0.16727495]


위 결과를 보면 카운트 벡터 값과 TF-IDF 벡터의 값이 많이 차이가 나는 것을 볼 수 있다. 그러나 아주 당연하게도 카운트 값이 0인 것은 TF-IDF 값도 여전히 0이 된다.

다음은 카운트 벡터를 거치지 않고 처음부터 바로 TF-IDF 행렬을 생성하는 예시다. 기왕 하는 김에 TF-IDF에 대해서도 코사인 유사도를 구해본다. 비교를 위해 여기서도 word_features를 사용해 생성한다.

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

start = len(reviews[0]) // 2
source = reviews[0][-start:]
source_cv = cv.transform([source])
sim_result = cosine_similarity(source_cv, reviews_cv) 

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

tf = TfidfVectorizer(vocabulary=word_features)
reviews_tf = tf.fit_transform(reviews)

# 코사인 유사도는 카운트 벡터에 대해 계산하므로 벡터로 변환
# transform은 반드시 리스트나 행렬 형태의 입력을 요구하므로 리스트로 만들어서 입력
source_tf = tf.transform([source])

# 변환된 count vector와 기존 값들과의 similarity 계산
sim_result_tf = cosine_similarity(source_tf, reviews_tf)

print("# 가장 유사한 리뷰의 인덱스:", np.argmax(sim_result_tf[0]))

# 가장 유사한 리뷰의 인덱스: 0


위 결과를 보면 TF-IDF에서도 원본을 잘 찾아내는 것을 볼 수 있다. 그렇다면 원본을 제외한 나머지 유사한 리뷰들은 과연 동일할까? 이를 확인하기 위해 아래와 같이 코사인 유사도가 높은 값들부터 20개씩 출력해서 비교해본다. 

In [12]:
print(
    "# 카운트 벡터에 대해 가장 유사한 리뷰부터 정렬한 인덱스:",
    (-sim_result[0]).argsort()[:10]
)

print(
    "# TF-IDF 벡터에 대해 가장 유사한 리뷰부터 정렬한 인덱스:",
    (-sim_result_tf[0]).argsort()[:10]
)

# 카운트 벡터에 대해 가장 유사한 리뷰부터 정렬한 인덱스: [   0 1110 1570  687  628  112 1712 1393  524 1740]
# TF-IDF 벡터에 대해 가장 유사한 리뷰부터 정렬한 인덱스: [   0 1110 1393 1570  645  323 1143  628 1676 1391]


위 결과를 보면 0을 제외하고 1110, 1570, 1393, 628이 양쪽에 공통으로 들어갔지만 순위는 0과 1110을 제외하고는 모두 달라진 것을 볼 수 있다. 이러한 차이는 이후 문서 분류와 같은 머신러닝 작업에서도 성능의 차이를 만들어낸다. 둘 중 어느 것이 더 정확한 유사도를 제공하는가에 대하여, 일반적으로는 TF-IDF가 더 정확한 것으로 알려져 있다. 그러나 용도에 따라 달라질 수 있으므로 논리적인 추론과 실험을 통해 선택할 필요가 있다.